##  감성체인 생성

In [1]:
import os
import time
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain.chains import LLMChain  # LLMChain을 사용하여 Chain을 구성합니다
import logging

In [2]:
# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

In [8]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI LLM 모델 설정
llm = ChatOpenAI(openai_api_key=api_key, model_name="gpt-4-turbo-preview", temperature=0.1)

# 감성 분석 기본 프롬프트
prompt_template = PromptTemplate.from_template(
    """ You are a Chinese social scientist, and your task is to analyze Chinese news reports about the "US-Japan-South Korea" trilateral relationship and perform sentiment analysis.

    **Scoring Criteria**:
    - Strongly Negative (-5): Extremely negative
    - Negative (-3 to -4): Rather negative
    - Slightly Negative (-1 to -2): Slightly negative
    - Neutral (0): Purely factual, no emotional bias
    - Slightly Positive (+1 to +2): Slightly positive
    - Positive (+3 to +4): Rather positive
    - Strongly Positive (+5): Extremely positive

    **Please strictly analyze each sentence of the following news article and provide a sentiment score for each sentence. All sentences provided must be analyzed; none should be omitted.**
    **Sentences should be separated based on the full stop (。) punctuation.**
    **News article content:**
    {article_content}

    **Please return the sentiment score for each sentence, one per line, with no additional text, annotations, or explanations.**
    """
)

In [9]:
# 체인 구성
sentiment_chain = LLMChain(prompt=prompt_template, llm=llm)

## 데이터 로드

In [5]:
import json

# JSON 파일 로드
file_path = "../data/0306/smaple.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"총 {len(data)}개의 기사 로드 완료")


총 6개의 기사 로드 완료


## 감성분석 수행 함수

In [6]:
# 감성 분석 수행 함수
def analyze_sentiment(article):
    """기사 전체 본문을 입력하고 문장별 감성 점수를 분석하는 함수"""
    time.sleep(1.5)  # Rate Limit 방지를 위해 대기
    try:
        # 기사 본문만 전달
        input_data = {"article_content": article["cleaned_content"]}

        # `sentiment_chain.run()`을 사용하여 데이터 처리
        response = sentiment_chain.invoke(input_data)  # input_data 전달

        print(f"Response format: {response}")

        # 'text' 키에서 감성 분석 결과를 추출하여 'sentiment_responses'에 저장
        if isinstance(response, dict):
            if 'text' in response:
                article["sentiment_responses"] = response['text'].strip()  # 감성 분석 결과 저장
            else:
                article["sentiment_responses"] = "Error: 'text' key not found in response"
        else:
            article["sentiment_responses"] = "Error: Response is not a dictionary"

        return article
    except Exception as e:
        article["sentiment_responses"] = f"Error: {str(e)}"  # 오류 발생 시 오류 메시지 저장
        return article  # 오류 발생 시에도 응답 내용 저장

def process_article(article):
    """GPT 응답 후처리"""
    sentiment_response = article.get("sentiment_responses", "")

    # 문장이 없는 경우 바로 반환
    if not sentiment_response:
        return article

    sentiment_scores = []
    for line in sentiment_response.split("\n"):
        try:
            # 문장별 감성 점수를 처리
            sentiment_scores.append(float(line.strip()))  # 각 점수 추가
        except (ValueError, IndexError):
            continue
    article["sentiment_scores"] = sentiment_scores
    article["avg_sentiment_score"] = round(sum(sentiment_scores) / len(sentiment_scores), 2) if sentiment_scores else None
    return article

def run_sentiment_analysis(data, output_file):
    """JSON 파일을 로드하여 감성 분석을 병렬 실행하고 결과를 저장하는 함수"""
    results = []

    # 기사별 병렬 실행 (최대 3개 스레드 사용)
    with ThreadPoolExecutor(max_workers=3) as executor:
        future_to_article = {executor.submit(analyze_sentiment, article): article for article in data}

        for future in as_completed(future_to_article):
            article = future_to_article[future]  # 원본 기사 가져오기
            processed_article = process_article(article)  # 후처리
            if processed_article is not None:
                results.append(processed_article)  # 결과 저장

            # 중간 저장 (기사별로 즉시 JSON 파일 업데이트)
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(results, f, ensure_ascii=False, indent=4)
            print(f" '{article['title']}' 분석 완료.")
    print(f" 전체 기사 감성 분석 완료. {output_file}")

In [10]:
output_path = "../data/0306/results_영어_온도1.json"
run_sentiment_analysis(data, output_path)

2025-03-06 14:49:42,065 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-06 14:49:42,103 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'article_content': '环球网记者王欢报道 10月11日上午，正在韩国访问的日本民主党政调会长前原诚司在首尔发表演讲。在大谈日韩两国应该签署“日韩经济合作协定”（EPA）之后，这位被视为对华强硬的日本“鹰派”政客再次提及中国，宣称美日韩三国并非要“封杀”中国，而要“敦促”中国“提高遵守国际准则的认识”。复旦大学教授冯玮对环球网记者表示，前原提及所谓“国际准则”问题，实际上是要以此混淆中国在领土问题上的立场。 据日本《产经新闻》10月11日消息，前原在演讲中表示，日韩签署EPA后，将会以此为契机，深化两国之间的情意、加强经济及人文交流，最终促进两国的共同繁荣；而日本文化的开放使得在日出现了“韩流热潮”，因此签署EPA对韩方而言有很多优点。他还说：“日韩两国是拥有共同价值观的友好邻邦，也是有着极其重要战略合作目标的伙伴。” 报道称，围绕日韩缔结EPA问题，日外相玄叶光一郎曾于6-7日与韩外交通商部部长金星焕就重启谈判进行磋商，但韩方担心协定签署后对日贸易赤字将会进一步扩大，因此一直犹豫不决。', 'text': '0\n-1\n-2\n+2\n+3\n-1\n-2'}
 '前原诚司在韩国发表演讲称美日韩并非要“封杀”中国 - 国际新闻' 분석 완료.
Response format: {'article_content': '据共同社6月21日报道，日本海上自卫队同美韩两国海军的首次联合军演21日在朝鲜半岛以南的公海拉开帷幕。部署在美海军横须贺基地的核动力航母“乔治·华盛顿”首次参加军演。本次联合军演为期两天。 据美国国防部等称，三国此前曾在美国夏威夷海域及日本海进行过数次联合搜救训练，但有美国航母及三国舰艇参加的正式军演尚属首次。 美国政府基于重视亚太地区的新国防战略，致力于推进同该地区盟国及友好国家的合作关系。 日本海上自卫队派出宙斯盾舰“雾雨”和搭载直升机的大型护卫舰“鞍马”等3艘舰艇参加训练。本次军演旨在提升三国的携同作战水平及通信能力，以加强确保海上安全及救灾等领域的合作。 另一方面，由于日韩两国存在殖民地统治等历史背景，韩国国内部分舆论对日本舰艇在周边海域进行军演表示忧虑。韩国国防部称此次军演“是定期训练”，未公布有关参加舰艇等详细情况，也未正式承认有美国航母参加。', 'text': '0\n0\n0\n

2025-03-06 14:49:43,669 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'article_content': '朝鲜国防委员会１６日提议与美国举行政府间高级别会谈之后，美国、日本和韩国分别回应提议。 要看行动 朝鲜国防委员会发言人１６日说，为消除朝鲜半岛紧张局面，确保地区和平与安全，建议朝美当局举行高级别会谈。朝方强调，不应给对话和接触设前提。 就朝方提议，美国白宫办公厅主任丹尼斯·麦克多诺１６日在哥伦比亚广播公司“面对全国”访谈节目中说：“这些会谈必须真实，必须依据他们（朝方）对包括不扩散、核武器、走私和其他问题在内的落实情况。” “因此，我们会通过他们的行动来判断，而不是我们昨天（美国东部时间１５日）听到的漂亮话，”麦克多诺说。 麦克多诺强调，底线是朝方不要试图利用谈判逃避正在受到的制裁。 麦克多诺与当天早些时候白宫国家安全委员会发言人凯特琳·海登的声明相呼应。海登在声明中说，美国愿意与朝鲜举行“可信的”谈判，但朝方必须履行其国际义务。 “我们一直赞成对话，实际上已经开通与朝鲜沟通的渠道，”她说，“但这些会谈必须伴随朝鲜对世界的义务，包括遵守联合国安理会决议，最终实现无核化。” 三方会谈 在朝方声明发出提议９小时之后，美国白宫作出回应，但美方没有明确表示是否接受这一提议。朝美是否就这一提议实现直接接触尚无法确认。 美国政府一名高级官员１６日透露，美方将于本周晚些时候与日本和韩国处理涉朝问题的高级外交官在华盛顿会晤期间讨论朝方提议。 这名不愿公开姓名的官员说，美方将与日本、韩国伙伴举行三方会谈，朝方提议将是讨论主题之一。朝鲜国防委员会１６日提议与美国举行政府间高级别会谈之后，美国、日本和韩国分别回应提议。 要看行动 朝鲜国防委员会发言人１６日说，为消除朝鲜半岛紧张局面，确保地区和平与安全，建议朝美当局举行高级别会谈。朝方强调，不应给对话和接触设前提。 就朝方提议，美国白宫办公厅主任丹尼斯·麦克多诺１６日在哥伦比亚广播公司“面对全国”访谈节目中说：“这些会谈必须真实，必须依据他们（朝方）对包括不扩散、核武器、走私和其他问题在内的落实情况。 因此，我们会通过他们的行动来判断，而不是我们昨天（美国东部时间１５日）听到的漂亮话，”麦克多诺说。 麦克多诺强调，底线是朝方不要试图利用谈判逃避正在受到的制裁。 麦克多诺与当天早些时候白宫国家安全委员会发言人凯特琳·海登的声明相呼应。海登在声明中说，美国愿意与朝鲜举

2025-03-06 14:49:45,020 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'article_content': '美国国务院高级官员26日表明有意下周在泰国首都曼谷举行美国、日本、韩国外交部长会议，以推动缓和日韩紧张关系。 美国国务卿迈克·蓬佩奥定于31日抵达曼谷，8月2日出席东盟地区论坛。日本外务大臣河野太郎、韩国外交部长官康京和同期赴曼谷参会。 一名不愿公开姓名的美国国务院高级官员26日告诉媒体记者：“我认为每次美国、韩国、日本（代表）在同一个地方时，会有聚一聚的想法。” 他说，美国政府“关切”日韩紧张关系，担心日韩不和影响三国就朝鲜半岛无核化进程合作。美方“正在寻找方法，以推动（日韩）双方以有效互惠方式消除对立”。 美日韩外长去年7月会面，蓬佩奥当时访朝结束，在日本首都东京与日韩两国外长会面。今年6月大阪二十国集团领导人会议期间，没有举行三方外长会。 美日韩三国国会议员26日在美国首都华盛顿闭门会商改善日韩关系。韩方议员要求日方取消对韩出口管制，认定管制举措关联韩国劳工向日本企业索赔。日方否认以贸易手段报复韩方，坚持限制出口合理。 出席会商的议员告诉媒体记者，日韩议员同意推动各自政府改善两国关系。韩国打算近期派遣国会代表团赴日。（刘秀玲）【新华社微特稿】', 'text': '0\n0\n0\n-1\n-2\n0\n-1\n-1\n-1\n+1\n+1'}
 '日美韩外长拟下周开会以推动缓和日韩紧张关系' 분석 완료.


2025-03-06 14:49:45,789 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'article_content': '新华社华盛顿１０月１０日电（记者刘晨 陆佳飞）美国军方１０日说，美军两架战略轰炸机当晚与日本和韩国战斗机在日本海附近进行了飞行军事演练。 美军太平洋空军在一份声明中说，美空军两架Ｂ－１Ｂ轰炸机当天从关岛美军基地出发完成了此次飞行。日本航空自卫队和韩国空军分别派出两架Ｆ－１５战斗机和两架Ｆ－１５Ｋ战斗机参加演练。 声明说，这是美军太平洋司令部Ｂ－１Ｂ轰炸机首次和日韩空军进行联合夜间飞行演练。此次演练意在展现位于亚太地区的美军有能力保护美国本土，同时也体现了美国与日韩间坚定的同盟关系。 朝鲜半岛局势持续紧张，美军机近期在东北亚活动频繁。９月２３日，美军多架战略轰炸机在战斗机的护航下至朝鲜以东水域上空飞行，抵达本世纪以来美军战机飞抵的韩朝非军事区最北端。', 'text': '0\n0\n0\n+1\n-2\n-1'}
 '美日韩军机在日本海附近进行夜间飞行演练' 분석 완료.


2025-03-06 14:49:47,733 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'article_content': '英国《金融时报》6月6日刊文《安倍应把竹岛主权交给韩国》，文章认为，日本与多国存在领土争端，而日本首相安倍晋三对日本历史行为所秉持的修正主义态度令邻国担忧。作者大胆假设，假如安倍把日韩之间具有主权争议的竹岛（韩国称独岛）交还韩国，对于日本乃至东亚都将具有积极的战略意义。主要内容编译如下： 安倍晋三唤醒了太平洋地区的民族主义幽灵。这位日本首相对日本历史行为所秉持的修正主义态度，令邻国感到担忧。中日关系由此受到的冲击，往往是西方媒体最为关注的。但日韩之间的火药味也越来越浓。在这方面，安倍能够且应当采取大胆的行动，以在历史上留下浓重的一笔，并促成该地区的彻底改观。 竹岛（Takeshima，韩国称“独岛”(Dokdo)）是日本海(Sea of Japan)中露出海面的几块“岩石”。韩国民族主义者认为，该群岛在日本帝国结束对朝鲜半岛的35年占领后已移交给韩国。日本政府当然不认同这种看法。安倍不应纠结于这个问题。他应该把该群岛的主权交给韩国。 日本一些历史学家最近就日本对竹岛（独岛）主权的主张提出质疑。比如说，韩国报纸《朝鲜日报》(Chosun Ilbo)最近引用了日本教授久保井规夫(Norio Kuboi)和黑田伊彦(Yoshihiro Kurod)的话，这两位教授是这样说的：“我们认为，独岛问题是一个历史问题，而非领土问题。”他们补充表示，在18世纪的日本地图上，独岛（在1904-05年日俄战争期间，日本出于战事需要占领了该群岛）在韩国主权范围内。韩国历史学家正确地指出，许多早期的日本地图并没有显示独岛是日本领土。 如果日本真的允许对历史记录作出不同的解读，那么日本政府摆出一个“高姿态”、把竹岛（独岛）让给韩国，将明白无误地表达出善意。此举的意义不亚于1977年安瓦尔 萨达特(Anwar Sadat)访问以色列、或1972年理查德 尼克松(Richard Nixon)访华。 考虑一下此举的轰动性影响：它将彻底改变韩国对日本的看法。安倍将成为诺贝尔和平奖(Nobel Peace Prize)最有力的争夺者。这一姿态还会迫使中国等对日本在19世纪和20世纪所作所为持负面看法的国家重新审视自己的看法。对日本来说，这些小小的岛屿其实没有什么经济、政治和战略价值，反而是一个地缘政治包袱。 有人可能会说，这样